In [1]:
! nvidia-smi

Wed Apr 29 22:58:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.00    Driver Version: 418.87.00    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
|  0%   55C    P2    59W / 280W |   1302MiB / 11178MiB |      3%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:03:00.0 Off |                  N/A |
| 48%   57C    P2    57W / 280W |   2738MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [2]:
import json

from adat.attackers.masked_cascada import MaskedCascada
from adat.models.masked_lm import MaskedLanguageModel
from adat.utils import calculate_wer, visualize_simple_diff

2020-04-29 22:58:50,856 [45938] WARNING  py.warnings:99: [JupyterRequire] /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])

2020-04-29 22:58:50,861 [45938] WARNING  py.warnings:99: [JupyterRequire] /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])

2020-04-29 22:58:50,864 [45938] WARNING  py.warnings:99: [JupyterRequire] /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of nump

In [3]:
data = []

with open('data/wine_class/test.json') as file:
    for line in file.readlines():
        data.append(json.loads(line))

In [4]:
attacker = MaskedCascada(
    masked_lm_dir="logs/wine_lm/", 
    classifier_dir="logs/wine_classifier/",
    deep_levenshtein_dir="logs/wine_levenshtein/",
    alpha=2.0,
    lr=0.5,
    device=3
)

In [5]:
labels_mapping = {0: 'bad wine', 1: 'good wine'}

def attack(sequence, label_to_attack, max_steps=10, 
           thresh_drop=0.2, early_stopping=False, attacker=attacker):
    adversarial_output = attacker.attack(
        sequence, 
        label_to_attack=label_to_attack, 
        max_steps=10,
        thresh_drop=0.2,
        early_stopping=False
    )

    adversarial_sequence = adversarial_output.adversarial_sequence
    wer = adversarial_output.wer
    prob = adversarial_output.probability
    new_prob = adversarial_output.adversarial_probability

    print(f'Initial label: {labels_mapping[label_to_attack]}\nInitial probability = {prob:.3f}\nNew probability = {new_prob:.3f}\nWER = {wer}\n')

    visualize_simple_diff(sequence, adversarial_sequence)

In [9]:
idx = 17

attack(data[idx]['text'], data[idx]['label'])

Initial label: good wine
Initial probability = 0.952
New probability = 0.303
WER = 2



In [11]:
idx = 18

attack(data[idx]['text'], data[idx]['label'], max_steps=5)

Initial label: bad wine
Initial probability = 0.907
New probability = 0.108
WER = 14



In [12]:
idx = 19

attack(data[idx]['text'], data[idx]['label'], max_steps=5)

Initial label: bad wine
Initial probability = 0.989
New probability = 0.660
WER = 6



In [14]:
idx = 21

attack(data[idx]['text'], data[idx]['label'], max_steps=5)

Initial label: good wine
Initial probability = 0.614
New probability = 0.069
WER = 7



In [15]:
idx = 22

attack(data[idx]['text'], data[idx]['label'], max_steps=5)

Initial label: good wine
Initial probability = 0.944
New probability = 0.625
WER = 4



In [16]:
idx = 23

attack(data[idx]['text'], data[idx]['label'], max_steps=5)

Initial label: bad wine
Initial probability = 0.859
New probability = 0.001
WER = 41



In [18]:
idx = 23

attack(data[idx]['text'], data[idx]['label'], max_steps=1)

Initial label: bad wine
Initial probability = 0.859
New probability = 0.000
WER = 33



In [21]:
attacker = MaskedCascada(
    masked_lm_dir="logs/wine_lm/", 
    classifier_dir="logs/wine_classifier/",
    deep_levenshtein_dir="logs/wine_levenshtein/",
    alpha=1.0,
    lr=0.1
)

In [23]:
idx = 23

attack(data[idx]['text'], data[idx]['label'], max_steps=5)

Initial label: bad wine
Initial probability = 0.859
New probability = 0.552
WER = 2



In [29]:
idx = 26

attack(data[idx]['text'], data[idx]['label'], max_steps=10)

Initial label: bad wine
Initial probability = 0.986
New probability = 0.738
WER = 4

